In [ ]:
Задача

К нам пришли наши коллеги из ML-отдела и рассказали, что планируют выкатывать новый алгоритм, рекомендующий нашим пользователям интересные посты. 
После обсуждений того, как он это делает, вы пришли к следующему пониманию:

1. Алгоритм добавляет пользователям 1-2 просмотра
2. Вероятность того, что он сработает, составляет 90%
3. Если у пользователя меньше 50 просмотров, то алгоритм не сработает

Вы предполагаете, что увеличение числа просмотров приведёт и к увеличению лайков на пользователя. Встаёт вопрос: сможем ли мы обнаружить различия в среднем 
количестве лайков на пользователя? Чтобы ответить на этот вопрос, давайте проведём симуляцию Монте-Карло!

Что мы будем делать:

- Распределения, из которых мы будем симулировать просмотры и пользовательские CTR, мы построим на основе периода АА-теста (даты смотрите в прошлом уроке). 
Выгрузите данные запросами, которые использовались в лекции, но уберите всё, связанное с exp_group. Данные нам понадобятся целиком, а на агрегацию эта переменная всё равно не повлияет.
- На эксперимент нам выделили неделю. Допустим, что за эту неделю в наш сервис зайдёт столько же пользователей, сколько зашло в период АА-теста. Мы планируем разбивать пользователей 
на две группы в соотношении 50/50. Посчитайте, сколько пользователей в таком случае придётся на одну группу.
- Эффект алгоритма на просмотры мы сымитируем следующим образом: group_B_views + ((1 + np.binomial(n=1, p=0.5, size=размер_выборки)) * np.binomial(n=1, p=0.9, size=размер_выборки) * (group_B_views >= 50)). 
Внимательно изучите эту строчку кода и подумайте, как она соотносится с описанием эффекта выше.
- Количество симуляций задайте не меньше 20000. Если хотите ещё больше уверенности в своих результатах — можете увеличить их число, но без фанатизма. 
- Лайки мы будем сравнивать t-тестом с поправкой Уэлча на неравные дисперсии (equal_var=False). Уровень значимости по классике поставим 0.05.

In [2]:
# === Импорты ===
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

try:
    from tqdm import tqdm
    USE_TQDM = True
except ImportError:
    USE_TQDM = False

import pandahouse as ph

# === 1. Подключение к ClickHouse ===
connection = {
    'host': 'http://clickhouse.lab.karpov.courses:8123',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20251120'
}

# === 2. Параметры периода A/A-теста и эксперимента ===
AA_START = '2025-10-19'
AA_END   = '2025-10-25'

ALPHA = 0.05
N_EXPERIMENTS = 20000     # можно 30000–50000 для стабильности, если ноут тянет
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)

# === 3. Выгрузка исторических данных (views / likes по пользователю и дню) ===
# ВАЖНО: группируем по user_id И дню
query = f"""
SELECT
    user_id,
    toDate(time) AS event_date,
    countIf(action = 'view') AS views,
    countIf(action = 'like') AS likes
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AA_END}'
GROUP BY user_id, event_date
"""

df = ph.read_clickhouse(query, connection=connection)
print("Данных выгружено (user_id, day):", len(df))

# убираем строки без просмотров
df = df[df['views'] > 0].copy()
print("После фильтра по views > 0 осталось строк (user_id, day):", len(df))

# === 3.1. Распределение просмотров: недельные просмотры на пользователя ===
df_week = (
    df
    .groupby('user_id', as_index=False)
    .agg({'views': 'sum', 'likes': 'sum'})
)
print("Пользователей за период A/A:", len(df_week))

views_per_user = df_week['views'].to_numpy(dtype=int)

# === 3.2. Распределение CTR: дневной CTR пользователя ===
df['ctr_day'] = df['likes'] / df['views']
ctr_per_user_day = df['ctr_day'].to_numpy(float)

# === 4. Размер одной группы при разбиении 50/50 по пользователям ===
n_total_users = len(df_week)
n_group = n_total_users // 2
print("Всего пользователей:", n_total_users)
print("Пользователей на одну группу (50/50):", n_group)

# === 5. Монте-Карло-симуляция эффекта и t-тест по лайкам на пользователя ===
p_values = np.empty(N_EXPERIMENTS)

iterator = range(N_EXPERIMENTS)
if USE_TQDM:
    iterator = tqdm(iterator)

for i in iterator:
    # 5.1. Сэмплируем просмотры (недельные) и CTR (дневные) из эмпирических распределений
    # views — распределение по пользователям
    views_A = rng.choice(views_per_user, size=n_group, replace=True)
    views_B = rng.choice(views_per_user, size=n_group, replace=True)

    # CTR — распределение по (user_id, day), чтобы учесть дневные вариации
    ctr_A = rng.choice(ctr_per_user_day, size=n_group, replace=True)
    ctr_B = rng.choice(ctr_per_user_day, size=n_group, replace=True)

    # 5.2. Эффект алгоритма на просмотры в группе B
    #   - добавляет 1–2 просмотра
    #   - срабатывает с вероятностью 0.9
    #   - не работает, если views < 50
    extra_views_base = 1 + rng.binomial(n=1, p=0.5, size=n_group)   # 1 или 2
    algo_trigger     = rng.binomial(n=1, p=0.9, size=n_group)       # 0 или 1
    eligible         = (views_B >= 50).astype(int)                  # 0 или 1

    extra_views = extra_views_base * algo_trigger * eligible
    new_views_B = views_B + extra_views

    # 5.3. Генерируем лайки по биномиальному распределению
    likes_A = rng.binomial(n=views_A,     p=ctr_A)
    likes_B = rng.binomial(n=new_views_B, p=ctr_B)

    # 5.4. t-тест Уэлча по лайкам на пользователя
    stat, p = ttest_ind(likes_B, likes_A, equal_var=False)
    p_values[i] = p

# === 6. Оценка мощности ===
power = (p_values < ALPHA).mean()
print(f"\nОценка мощности t-теста (доля симуляций с p < {ALPHA}): {power:.4f}")
print(f"Мощность t-теста в процентах: {power * 100:.1f}")


Данных выгружено (user_id, day): 85121
После фильтра по views > 0 осталось строк (user_id, day): 85121
Пользователей за период A/A: 41997
Всего пользователей: 41997
Пользователей на одну группу (50/50): 20998


100%|██████████| 20000/20000 [03:22<00:00, 98.86it/s] 


Оценка мощности t-теста (доля симуляций с p < 0.05): 0.2635
Мощность t-теста в процентах: 26.3


In [ ]:
К нам снова пришли коллеги из ML-отдела с радостной новостью: они улучшили качество алгоритма! 
Теперь он срабатывает на пользователях с числом просмотров от 30 и выше.

In [3]:
# === Импорты ===
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

try:
    from tqdm import tqdm
    USE_TQDM = True
except ImportError:
    USE_TQDM = False

import pandahouse as ph

# === 1. Подключение к ClickHouse ===
connection = {
    'host': 'http://clickhouse.lab.karpov.courses:8123',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20251120'
}

# === 2. Параметры периода A/A-теста и эксперимента ===
AA_START = '2025-10-19'
AA_END   = '2025-10-25'

ALPHA = 0.05
N_EXPERIMENTS = 20000     # можно поднять, если нужно
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)

# === 3. Выгрузка исторических данных (views / likes по пользователю и дню) ===
query = f"""
SELECT
    user_id,
    toDate(time) AS event_date,
    countIf(action = 'view') AS views,
    countIf(action = 'like') AS likes
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AA_END}'
GROUP BY user_id, event_date
"""

df = ph.read_clickhouse(query, connection=connection)
print("Данных выгружено (user_id, day):", len(df))

# убираем строки без просмотров
df = df[df['views'] > 0].copy()
print("После фильтра по views > 0 осталось строк (user_id, day):", len(df))

# === 3.1. Распределение просмотров: недельные просмотры на пользователя ===
df_week = (
    df
    .groupby('user_id', as_index=False)
    .agg({'views': 'sum', 'likes': 'sum'})
)
print("Пользователей за период A/A:", len(df_week))

views_per_user = df_week['views'].to_numpy(dtype=int)

# === 3.2. Распределение CTR: дневной CTR пользователя ===
df['ctr_day'] = df['likes'] / df['views']
ctr_per_user_day = df['ctr_day'].to_numpy(float)

# === 4. Размер одной группы при разбиении 50/50 по пользователям ===
n_total_users = len(df_week)
n_group = n_total_users // 2
print("Всего пользователей:", n_total_users)
print("Пользователей на одну группу (50/50):", n_group)

# === 5. Монте-Карло-симуляция эффекта и t-тест по лайкам на пользователя ===
p_values = np.empty(N_EXPERIMENTS)

iterator = range(N_EXPERIMENTS)
if USE_TQDM:
    iterator = tqdm(iterator)

for i in iterator:
    # 5.1. Сэмплируем просмотры (недельные) и CTR (дневные) из эмпирических распределений
    views_A = rng.choice(views_per_user, size=n_group, replace=True)
    views_B = rng.choice(views_per_user, size=n_group, replace=True)

    ctr_A = rng.choice(ctr_per_user_day, size=n_group, replace=True)
    ctr_B = rng.choice(ctr_per_user_day, size=n_group, replace=True)

    # 5.2. Эффект алгоритма на просмотры в группе B
    #   - добавляет 1–2 просмотра
    #   - срабатывает с вероятностью 0.9
    #   - теперь работает, если views >= 30 (было 50)
    extra_views_base = 1 + rng.binomial(n=1, p=0.5, size=n_group)   # 1 или 2
    algo_trigger     = rng.binomial(n=1, p=0.9, size=n_group)       # 0 или 1
    eligible         = (views_B >= 30).astype(int)                  # ВАЖНО: порог 30

    extra_views = extra_views_base * algo_trigger * eligible
    new_views_B = views_B + extra_views

    # 5.3. Генерируем лайки по биномиальному распределению
    likes_A = rng.binomial(n=views_A,     p=ctr_A)
    likes_B = rng.binomial(n=new_views_B, p=ctr_B)

    # 5.4. t-тест Уэлча по лайкам на пользователя
    stat, p = ttest_ind(likes_B, likes_A, equal_var=False)
    p_values[i] = p

# === 6. Оценка мощности ===
power = (p_values < ALPHA).mean()
print(f"\nОценка мощности t-теста (доля симуляций с p < {ALPHA}): {power:.4f}")
print(f"Мощность t-теста в процентах: {power * 100:.1f}")


Данных выгружено (user_id, day): 85121
После фильтра по views > 0 осталось строк (user_id, day): 85121
Пользователей за период A/A: 41997
Всего пользователей: 41997
Пользователей на одну группу (50/50): 20998


100%|██████████| 20000/20000 [03:19<00:00, 100.04it/s]


Оценка мощности t-теста (доля симуляций с p < 0.05): 0.4273
Мощность t-теста в процентах: 42.7


In [ ]:
Теперь нас пришло радовать начальство: нам утвердили длительность эксперимента длиной в 2 недели! Давайте теперь допустим, что в эти две недели к нам придёт столько же пользователей, 
сколько пришло суммарно за период АА-теста и АБ-теста (опять же, смотрите диапазон дат в прошлом уроке).

Подкорректируйте размер группы соответствующим образом. Сами распределения, на основе которых мы симулируем данные, менять не будем.

In [5]:
# === Импорты ===
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

try:
    from tqdm import tqdm
    USE_TQDM = True
except ImportError:
    USE_TQDM = False

import pandahouse as ph

# === 1. Подключение к ClickHouse ===
connection = {
    'host': 'http://clickhouse.lab.karpov.courses:8123',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20251120'
}

# === 2. Периоды ===
AA_START = '2025-10-19'
AA_END   = '2025-10-25'

AB_START = '2025-10-26'
AB_END   = '2025-11-01'

ALPHA = 0.05
N_EXPERIMENTS = 20000     # можно поднять для более стабильной оценки
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)

# === 3. Распределения для симуляции: считаем ТОЛЬКО по A/A-периоду ===
#   - views_per_user: недельные просмотры на пользователя
#   - ctr_per_user_day: дневной CTR пользователя (user, day)

query_aa = f"""
SELECT
    user_id,
    toDate(time) AS event_date,
    countIf(action = 'view') AS views,
    countIf(action = 'like') AS likes
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AA_END}'
GROUP BY user_id, event_date
"""

df = ph.read_clickhouse(query_aa, connection=connection)
print("A/A-период: строк (user_id, day):", len(df))

df = df[df['views'] > 0].copy()
print("После фильтра по views > 0:", len(df))

# недельные просмотры/лайки на пользователя
df_week = (
    df
    .groupby('user_id', as_index=False)
    .agg({'views': 'sum', 'likes': 'sum'})
)
print("Пользователей в A/A:", len(df_week))

views_per_user = df_week['views'].to_numpy(dtype=int)

# дневной CTR пользователя
df['ctr_day'] = df['likes'] / df['views']
ctr_per_user_day = df['ctr_day'].to_numpy(float)

# === 4. Размер групп для 2-недельного эксперимента ===
# Берём КОЛ-ВО уникальных пользователей за A/A + A/B и делим пополам

query_users = f"""
SELECT
    user_id
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AB_END}'
GROUP BY user_id
"""

df_users = ph.read_clickhouse(query_users, connection=connection)
n_total_users_exp = len(df_users)
n_group = n_total_users_exp // 2

print("Уникальных пользователей за A/A + A/B:", n_total_users_exp)
print("Пользователей на одну группу (50/50):", n_group)

# === 5. Монте-Карло-симуляция эффекта и t-тест по лайкам на пользователя ===
p_values = np.empty(N_EXPERIMENTS)

iterator = range(N_EXPERIMENTS)
if USE_TQDM:
    iterator = tqdm(iterator)

for i in iterator:
    # 5.1. Сэмплируем просмотры (недельные) и CTR (дневные) из эмпирических распределений A/A
    views_A = rng.choice(views_per_user, size=n_group, replace=True)
    views_B = rng.choice(views_per_user, size=n_group, replace=True)

    ctr_A = rng.choice(ctr_per_user_day, size=n_group, replace=True)
    ctr_B = rng.choice(ctr_per_user_day, size=n_group, replace=True)

    # 5.2. Эффект алгоритма на просмотры в группе B
    #   - добавляет 1–2 просмотра
    #   - срабатывает с вероятностью 0.9
    #   - работает, если views >= 30
    extra_views_base = 1 + rng.binomial(n=1, p=0.5, size=n_group)   # 1 или 2
    algo_trigger     = rng.binomial(n=1, p=0.9, size=n_group)       # 0 или 1
    eligible         = (views_B >= 30).astype(int)                  # порог 30

    extra_views = extra_views_base * algo_trigger * eligible
    new_views_B = views_B + extra_views

    # 5.3. Генерируем лайки по биномиальному распределению
    likes_A = rng.binomial(n=views_A,     p=ctr_A)
    likes_B = rng.binomial(n=new_views_B, p=ctr_B)

    # 5.4. t-тест Уэлча по лайкам на пользователя
    stat, p = ttest_ind(likes_B, likes_A, equal_var=False)
    p_values[i] = p

# === 6. Оценка мощности ===
power = (p_values < ALPHA).mean()
print(f"\nОценка мощности t-теста (доля симуляций с p < {ALPHA}): {power:.4f}")
print(f"Мощность t-теста в процентах: {power * 100:.1f}")


A/A-период: строк (user_id, day): 85121
После фильтра по views > 0: 85121
Пользователей в A/A: 41997
Уникальных пользователей за A/A + A/B: 61182
Пользователей на одну группу (50/50): 30591


100%|██████████| 20000/20000 [04:43<00:00, 70.56it/s]


Оценка мощности t-теста (доля симуляций с p < 0.05): 0.5721
Мощность t-теста в процентах: 57.2


In [ ]:
Всё это время мы анализировали наши выборки целиком — и тех пользователей, на которых алгоритм повлиял, и тех, кого он не мог затронуть (меньше 30 просмотров). 
А что, если мы будем отбирать только нужных пользователей и скармливать t-тесту именно их? Да, выборка будет меньше, но мы избавимся от мусора — а значит, 
и чувствительность наверняка будет выше.

In [6]:
# === Импорты ===
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

try:
    from tqdm import tqdm
    USE_TQDM = True
except ImportError:
    USE_TQDM = False

import pandahouse as ph

# === 1. Подключение к ClickHouse ===
connection = {
    'host': 'http://clickhouse.lab.karpov.courses:8123',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20251120'
}

# === 2. Периоды ===
AA_START = '2025-10-19'
AA_END   = '2025-10-25'

AB_START = '2025-10-26'
AB_END   = '2025-11-01'

ALPHA = 0.05
N_EXPERIMENTS = 20000     # можно поднять для более стабильной оценки
RANDOM_SEED = 42

rng = np.random.default_rng(RANDOM_SEED)

# === 3. Распределения для симуляции: считаем ТОЛЬКО по A/A-периоду ===
#   - views_per_user: недельные просмотры на пользователя
#   - ctr_per_user_day: дневной CTR пользователя (user, day)

query_aa = f"""
SELECT
    user_id,
    toDate(time) AS event_date,
    countIf(action = 'view') AS views,
    countIf(action = 'like') AS likes
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AA_END}'
GROUP BY user_id, event_date
"""

df = ph.read_clickhouse(query_aa, connection=connection)
print("A/A-период: строк (user_id, day):", len(df))

df = df[df['views'] > 0].copy()
print("После фильтра по views > 0:", len(df))

# недельные просмотры/лайки на пользователя
df_week = (
    df
    .groupby('user_id', as_index=False)
    .agg({'views': 'sum', 'likes': 'sum'})
)
print("Пользователей в A/A:", len(df_week))

views_per_user = df_week['views'].to_numpy(dtype=int)

# дневной CTR пользователя
df['ctr_day'] = df['likes'] / df['views']
ctr_per_user_day = df['ctr_day'].to_numpy(float)

# === 4. Размер групп для 2-недельного эксперимента ===
# Берём КОЛ-ВО уникальных пользователей за A/A + A/B и делим пополам

query_users = f"""
SELECT
    user_id
FROM simulator_20251120.feed_actions
WHERE toDate(time) BETWEEN '{AA_START}' AND '{AB_END}'
GROUP BY user_id
"""

df_users = ph.read_clickhouse(query_users, connection=connection)
n_total_users_exp = len(df_users)
n_group = n_total_users_exp // 2

print("Уникальных пользователей за A/A + A/B:", n_total_users_exp)
print("Пользователей на одну группу (50/50):", n_group)

# === 5. Монте-Карло-симуляция эффекта и t-тест по лайкам на пользователя (только views >= 30) ===
p_values = np.empty(N_EXPERIMENTS)

iterator = range(N_EXPERIMENTS)
if USE_TQDM:
    iterator = tqdm(iterator)

for i in iterator:
    # 5.1. Сэмплируем просмотры (недельные) и CTR (дневные) из эмпирических распределений A/A
    views_A = rng.choice(views_per_user, size=n_group, replace=True)
    views_B = rng.choice(views_per_user, size=n_group, replace=True)

    ctr_A = rng.choice(ctr_per_user_day, size=n_group, replace=True)
    ctr_B = rng.choice(ctr_per_user_day, size=n_group, replace=True)

    # 5.2. Эффект алгоритма на просмотры в группе B
    #   - добавляет 1–2 просмотра
    #   - срабатывает с вероятностью 0.9
    #   - работает, если views >= 30
    extra_views_base = 1 + rng.binomial(n=1, p=0.5, size=n_group)   # 1 или 2
    algo_trigger     = rng.binomial(n=1, p=0.9, size=n_group)       # 0 или 1
    eligible         = (views_B >= 30).astype(int)                  # порог 30

    extra_views = extra_views_base * algo_trigger * eligible
    new_views_B = views_B + extra_views

    # 5.3. Генерируем лайки по биномиальному распределению
    likes_A = rng.binomial(n=views_A,     p=ctr_A)
    likes_B = rng.binomial(n=new_views_B, p=ctr_B)

    # 5.4. Фильтруем только пользователей, у которых базовые просмотры >= 30
    mask_A = views_A >= 30
    mask_B = views_B >= 30

    # На всякий случай, если вдруг в какой-то симуляции в маске почти никого нет,
    # можно подстраховаться:
    if mask_A.sum() < 2 or mask_B.sum() < 2:
        p_values[i] = 1.0
        continue

    # 5.5. t-тест Уэлча только на "релевантных" пользователях
    stat, p = ttest_ind(likes_B[mask_B], likes_A[mask_A], equal_var=False)
    p_values[i] = p

# === 6. Оценка мощности ===
power = (p_values < ALPHA).mean()
print(f"\nОценка мощности t-теста (доля симуляций с p < {ALPHA}): {power:.4f}")
print(f"Мощность t-теста в процентах: {power * 100:.1f}")


A/A-период: строк (user_id, day): 85121
После фильтра по views > 0: 85121
Пользователей в A/A: 41997
Уникальных пользователей за A/A + A/B: 61182
Пользователей на одну группу (50/50): 30591


100%|██████████| 20000/20000 [05:00<00:00, 66.66it/s]


Оценка мощности t-теста (доля симуляций с p < 0.05): 0.6523
Мощность t-теста в процентах: 65.2


In [ ]:
Оценивала мощность А/B-теста для нового рекомендательного алгоритма с помощью Монте-Карло-симуляций на исторических данных. 
Показала, что 1 неделя эксперимента даёт мощность ≤ 40%, тогда как удлинение до 2 недель и фокус на пользователях с достаточной активностью (≥30 просмотров) 
повышает мощность до ~65%, что принципиально меняет надёжность выводов по эксперименту